<a href="https://colab.research.google.com/github/marvinmouroum/AccountingApp/blob/master/Hacker's_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Part
... Done by **Harry** & **Lauri**

In [ ]:
!pip install patool

     |████████████████████████████████| 77 kB 5.4 MB/s 


In [ ]:
!rm -r train_data train_labels test_data test_labels

In [ ]:
from google.colab import drive
import glob 
import patoolib
import os
import shutil

# NEED TO OVERWRITE THE TRAIN LABELS AFTER EACH ITER
def overwrite_then_extract_archive(archive, verbosity=0, outdir=None, program=None):
  if outdir:
    if not os.listdir(outdir):
      if os.path.exists(outdir):
          shutil.rmtree(outdir)

  return patoolib.extract_archive(archive, verbosity, outdir, program)

# MONKEY PATCH
patoolib.extract_archive = overwrite_then_extract_archive 

class DataLoader:
  '''
  A class to load the data and organise for training

  Methods
  -------
  _mkdir: 
    make a directory
  _unrar: 
    extract a rar
  get_trainset_labels: 
    get the labels from training data
  get_transet: 
    get the input data from training data
  get_valset_labels: 
    get the labels from the validation data
  get_valset: 
    get the input data from the validation data
  '''

  def __init__(self, 
               dest_input, 
               dest_labels,
               dest_input_test,
               dest_labels_test, 
               verbosity=1):
    '''
    Parameters
    ----------
    dest : str 
      destination folder for data
    verbosity : int [0, 1, 2]
      display information for unrar package
    '''

    self.verbosity = verbosity
    self.dest_input = dest_input
    self.dest_labels = dest_labels
    self.dest_input_test = dest_input_test
    self.dest_labels_test = dest_labels_test
    self._mkdir()
    drive.mount('/content/drive')
    self.train_path = 'drive/MyDrive/Resources/train'
    self.train_labels_path = 'drive/MyDrive/Resources/train/labels.rar'
    self.train_files = glob.glob(self.train_path + '/*')[1:]

    self.val_path = 'drive/MyDrive/Resources/val'
    self.val_labels_path = 'drive/MyDrive/Resources/val/labels.rar'
    self.val_files = glob.glob(self.val_path + '/*')[1:]

  def _mkdir(self):
    ''' 
    Private make a directory if not made
    '''
    
    os.mkdir(self.dest_input)
    os.mkdir(self.dest_labels)
    os.mkdir(self.dest_input_test)
    os.mkdir(self.dest_labels_test)
    print('directories made')

  def _unrar(self, file_path, outdir):
    '''
    extract the rar files from the folder 
    Parameters
    ----------
    file path of rar file
    Returns
    -------
    result if it worked
    '''
    return patoolib.extract_archive(file_path, 
                                  outdir=outdir,
                                  verbosity=self.verbosity)
  
  def get_trainset_labels(self):
    '''
    Get the labels for the trainset

    Returns
    -------
    result if it worked
    '''
    return self._unrar(self.train_labels_path, self.dest_labels)

  def get_trainset(self):
    '''
    Generator to stream each rar file and serve
    images into designated file path
    
    returns
    -------
    iterator for extracting rar
    '''
    for file in self.train_files:
      yield self._unrar(file, self.dest_input)
  
  def get_valset_labels(self):
    '''
    Same thing as get_trainset_labels
    '''
    return self._unrar(self.val_labels_path, self.dest_labels_test)

  def get_valset(self):
    ''' 
    Same thing as get_transet
    '''
    for file in self.val_files:
      yield self._unrar(file, self.dest_input_test)


In [ ]:
loader = DataLoader('train_data', 'train_labels',
                    'test_data', 'test_labels')

directories made
Mounted at /content/drive


In [ ]:
!ls

drive  sample_data  test_data  test_labels  train_data	train_labels


In [ ]:
# just use for training. When running for val, run the DataLoader('val_input',
# 'val_labels'). Don't forget mkdirb, can't be f'ed to do it in class. Maybe l8r
# this takes so long

# DON'T FORGET, THE LABELS ARE FOR THE ENTIRE TRAIN SET, SO INDEX
res = loader.get_trainset_labels()
if res:
  for _ in loader.get_trainset():
    loader.prepare_directory('train')
    print("do training")

FileNotFoundError: ignored

## Loading a dataset

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
# PIL.Image.open('train_input/images/00000.png')

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'train_data',
  validation_split=0.2,
  subset="training",
  labels=''
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 467 files belonging to 2 classes.
Using 374 files for training.


## Dataset for Mask

In [ ]:
import os
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [ ]:
!mkdir unzipped_images
!mkdir unzipped_images/train
!mkdir unzipped_images/val

In [ ]:
directory = "drive/MyDrive/Resources/train"
for name in os.listdir(directory):
  print("on resource ", name)
  path = os.path.join(directory, name)
  !unrar x $path unzipped_images/train

The Labels Need to changed into Segmentation Masks

*Harry's Github code copy paste*

In [ ]:
import numpy as np 
from PIL import Image, ImageDraw

def data2mask(polygon: list, out_img_size: int) -> np.array: 
    '''
    Returns a mask from an 8 point polygon.
    :param polygon<list[int]>: list of 8 points, can be tuples.
    :param out_img_size<int>: creates an output image size (h == w)
    '''

    img = Image.new('L', (out_img_size, out_img_size), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    return np.array(img)


def file2data(filename: str, orig_img_size: int, out_img_size: int) -> list:
    '''
    Returns normalised data from the text file 
    :param filename<str>: source of the labels txt data
    :param orig_img_size<int>: the original size of the image (assuming x == y)
    :param out_img_size<int>: the size of the output image (x == y)
    '''

    with open(filename, 'r') as file:
        lines = file.readlines()
        data = [list(
                map(lambda x: int((x / orig_img_size)*(out_img_size)),
                    list(map(int, line.split('\n')[0].split(','))))) for line in lines]

    file.close()

    return data

def file2segmask(filename: str, orig_img_size:int, out_img_size:int) -> np.array:

    '''
    Runs the mask segmentation over a file: labels.txt.
    :param filename<str>: the filename within labels (00000.txt for example)
    :param orig_img_size<int>: size of the original image (assuming h==w)
    :param out_img_size<int>: size of the output image for training (h==w)
    '''

    # init
    out_mat: list = []

    # run
    data = file2data(filename, orig_img_size, out_img_size)
    for row in data:
        out_mat.append(data2mask(row, out_img_size))

    # return
    return np.array(out_mat)

In [ ]:
!ls unzipped_images/train

In [ ]:
!ls train_data/images

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 4
NUM_CLASSES = 1
DATA_DIR = "./instance-level_human_parsing/instance-level_human_parsing"
NUM_TRAIN_IMAGES = 1000
NUM_VAL_IMAGES = 50

train_images = sorted(glob(os.path.join(DATA_DIR, "Images/*")))[:NUM_TRAIN_IMAGES]
train_masks = sorted(glob(os.path.join(DATA_DIR, "Category_ids/*")))[:NUM_TRAIN_IMAGES]
val_images = sorted(glob(os.path.join(DATA_DIR, "Images/*")))[
    NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES
]
val_masks = sorted(glob(os.path.join(DATA_DIR, "Category_ids/*")))[
    NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES
]


def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = image / 127.5 - 1
    return image


def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask


def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset


train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

In [ ]:
train_ds

<BatchDataset shapes: ((None, 180, 180, 3), (None,)), types: (tf.float32, tf.int32)>

# Deeplab Part
...Done by **Marvin**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Model Definition

deeplabv3+

In [ ]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [ ]:

def set_backbone(name, model_input):

  backbone = None
  last_layer = None
  residual_layer = None
  devider = 1

  if name == "resnet":
    resnet50 = keras.applications.ResNet50(weights="imagenet", include_top=False, input_tensor=model_input)
    backbone = resnet50
    last_layer = "conv4_block6_2_relu"
    residual_layer = "conv2_block3_2_relu"
    devider = 4
  
  if name == "xception":
    xception = keras.applications.Xception(weights="imagenet", include_top=False, input_tensor=model_input)
    backbone = xception
    last_layer = "block13_sepconv1_act"
    residual_layer = "block4_sepconv1_act"
    devider = 8

  if name == "mobilenet":
    mobilenet = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=True, input_tensor=model_input)
    backbone = mobilenet
    last_layer = "block_15_depthwise_relu"
    residual_layer = "block_2_depthwise_relu"
    devider = 4

  return backbone, last_layer, residual_layer, devider

In [ ]:
model_input = keras.Input(shape=(256, 256, 3))
backbone, last_layer, residual_layer, n = set_backbone(name="mobilenet",model_input=model_input)

14548992/14536120 [==============================] - 0s 0us/step


In [ ]:
backbone.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
def DeeplabV3Plus(image_size, num_classes):
    
    model_input = keras.Input(shape=(image_size, image_size, 3))

    backbone, last_layer, residual_layer, devider = set_backbone(name="mobilenet",model_input=model_input)

    x = backbone.get_layer(last_layer).output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // devider // x.shape[1], image_size // devider // x.shape[2] ),
        interpolation="bilinear",
    )(x)
    input_b = backbone.get_layer(residual_layer).output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUMBER_CLASSES)
model.summary()

## Model Training

In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=["accuracy"],
)

history = model.fit(train_dataset, validation_data=val_dataset, epochs=25)

plt.plot(history.history["loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["accuracy"])
plt.title("Training Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_loss"])
plt.title("Validation Loss")
plt.ylabel("val_loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_accuracy"])
plt.title("Validation Accuracy")
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.show()

# Detection Part
.. Done by **Jonny**

# Ensemble

In [ ]:
def HackersEnsemble(image_size, num_classes, models:list):
  
  model_input = keras.Input(shape=(image_size, image_size, 3))

  return keras.Model(inputs=model_input, outputs=model_output)

# Pipeline

1. Fetch the Validation Data
2. Initialize the model
3. Start model evaluation

## Fetch the Validation Data

We need to get the validation data, unzip it plus the labels and prepare a Dataloader for feeding it to the network, that will be initialized in the next step

In [1]:
# Code for fetching the validation data

## Initialize the model

The model will be trained by that point so we just need to load it

In [2]:
# Code for initializing the model

## Start the evaluation

The evaluation code should assign the data to the model. And apply the evaluation criteria to it.

1. Measure the inference
2. measure the IoU

In [ ]:
# Code for starting the evaluation